In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sklearn
import matplotlib.pyplot as plt
#import plotly.graph_objs as go
#import plotly.offline as py
#import plotly.tools as tls
#from statsmodels.stats import weightstats as stests
from scipy import stats

import datetime
from datetime import datetime
import statistics as es 
import csv
import re 
import warnings
from apyori import apriori
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from IPython.display import Image  
from sklearn.tree import export_graphviz
from graphviz import Source
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from apyori import apriori


In [2]:
tweets=pd.read_csv('tweets.csv', sep=',')
tweets.drop("Unnamed: 0", axis=1, inplace=True)
users=pd.read_csv('users.csv', sep=',')
users.drop("Unnamed: 0", axis=1, inplace=True)

In [3]:
#transform id´s in object
tweets.user_id=tweets.user_id.astype(str)
users.user_id=users.user_id.astype(str)
print(users.user_id.dtypes)
print(tweets.user_id.dtypes)

object
object


In [4]:
#selecting columns. Droping some redundant features and unnecessary
columns_users=['user_id', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'url', 'location','default_profile'
        , 'profile_banner_url','profile_image_url_https' 
       ,'age','in_following_followers','n_number_sname',
       'size_sname', 'size_description', 'size_name', 'Alvo']
columns_tweets=['user_id','n_retweet_mean',
       'n_favorite_mean', 'n_hashtags_mean', 'n_urls_mean', 'n_mentions_mean',
       'media_size_twitter', 'dp_size_twitter', 'in_tweets_reply',
       'n_tweet_dia', 'max_count_tweet_dia', 'count_dias_tweet',
       'in_tweet_dia']

tweets.user_id = tweets.user_id.astype('float')
users.user_id = users.user_id.astype('float')

a = tweets[columns_tweets].drop_duplicates('user_id')
b = users[columns_users]
database=pd.merge(b, a, on='user_id', how='right')
database.drop('user_id', axis=1, inplace=True)
database.to_csv("users_tweets.csv", sep=',')

database.shape

(2682, 28)

In [6]:
database.dtypes

followers_count              int64
friends_count                int64
favourites_count             int64
listed_count                 int64
url                          int64
location                     int64
default_profile              int64
profile_banner_url           int64
profile_image_url_https      int64
age                          int64
in_following_followers     float64
n_number_sname               int64
size_sname                   int64
size_description             int64
size_name                    int64
Alvo                         int64
n_retweet_mean             float64
n_favorite_mean            float64
n_hashtags_mean            float64
n_urls_mean                float64
n_mentions_mean            float64
media_size_twitter         float64
dp_size_twitter            float64
in_tweets_reply              int64
n_tweet_dia                  int64
max_count_tweet_dia          int64
count_dias_tweet             int64
in_tweet_dia               float64
dtype: object

In [84]:
X = database.drop(['Alvo'], axis=1)
y = database['Alvo']

In [85]:
#Houldout
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [71]:
#decision_tree
model = DecisionTreeClassifier(max_depth=3, min_samples_leaf=20)
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [72]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9424379232505643

In [73]:
export_graphviz(model, out_file= "tree.dot", feature_names = list(X.columns),
                rounded = True, proportion = False, precision = 2, filled = True)
from subprocess import call
call(['dot', '-T', 'png', 'tree.dot', '-o', 'tree.png'])

0

## Handling outliers and Binnning data for better rendering

In [74]:
database.head(2)

,followers_count,friends_count,favourites_count,listed_count,url,location,default_profile,profile_banner_url,profile_image_url_https,age,in_following_followers,n_number_sname,size_sname,size_description,size_name,Alvo,n_retweet_mean,n_favorite_mean,n_hashtags_mean,n_urls_mean,n_mentions_mean,media_size_twitter,dp_size_twitter,in_tweets_reply,n_tweet_dia,max_count_tweet_dia,count_dias_tweet,in_tweet_dia
0,0,0,0,0,0,1,0,0,1,875,0.0,0,14,61,15,1,0.565,0.595,0.045,0.04,0.005,84.870,28.1935,0,1,21,94,2.127660
1,0,0,0,0,0,1,0,0,1,865,0.0,0,14,133,15,1,0.280,0.480,0.065,0.02,0.000,87.805,27.3811,0,3,18,104,1.923077


In [75]:
#context
def bin_age(df):
    col="age"
    if df[col]  <90:
        return 1
    elif df[col] <180:
        return 2
    elif df[col] <360:
        return 3
    elif df[col] <720:
        return 4
    elif df[col] <1080:
        return 5
    else: 
        return 6

In [76]:
database["age"]= database.apply(bin_age, axis=1)
database["age"].value_counts()

5    1119
6     879
4     511
2      88
1      46
3      39
Name: age, dtype: int64

In [77]:
#context
def bin_listed_count(df):
    col="listed_count"
    if df[col]  == 0:
        return 0
    else: 
        return 1

In [62]:
database["listed_count"]= database.apply(bin_listed_count, axis=1)
database["listed_count"].value_counts()

1    1816
0     866
Name: listed_count, dtype: int64

In [63]:
#save dataset with only two numerical features discretized by context.


In [45]:
numericals= ['in_following_followers',"n_number_sname",'n_retweet_mean',
       'n_favorite_mean', 'n_hashtags_mean', 'n_urls_mean', 'n_mentions_mean',
       'media_size_twitter', 'dp_size_twitter', 'in_tweets_reply',
       'n_tweet_dia', 'max_count_tweet_dia', 'count_dias_tweet',
       'in_tweet_dia','followers_count', 'friends_count','favourites_count'
        ,'n_number_sname','size_sname', 'size_description', 'size_name' ]



In [46]:
kbd = KBinsDiscretizer(n_bins= 4, encode= 'ordinal', strategy = 'quantile')
database[numericals]= kbd.fit_transform (database[numericals])


In [47]:
database.shape

(2682, 28)

In [48]:
for i in numericals:

    print(database[i].value_counts())


3.0    671
0.0    671
2.0    670
1.0    670
Name: in_following_followers, dtype: int64
0.0    2682
Name: n_number_sname, dtype: int64
3.0    671
2.0    671
1.0    671
0.0    669
Name: n_retweet_mean, dtype: int64
1.0    706
3.0    676
2.0    666
0.0    634
Name: n_favorite_mean, dtype: int64
2.0    706
3.0    674
0.0    659
1.0    643
Name: n_hashtags_mean, dtype: int64
2.0    686
1.0    681
3.0    675
0.0    640
Name: n_urls_mean, dtype: int64
0.0    1336
2.0     680
1.0     666
Name: n_mentions_mean, dtype: int64
0.0    671
3.0    671
1.0    670
2.0    670
Name: media_size_twitter, dtype: int64
0.0    671
3.0    671
1.0    670
2.0    670
Name: dp_size_twitter, dtype: int64
0.0    2009
1.0     673
Name: in_tweets_reply, dtype: int64
0.0    1316
1.0     693
2.0     673
Name: n_tweet_dia, dtype: int64
3.0    802
2.0    678
1.0    654
0.0    548
Name: max_count_tweet_dia, dtype: int64
3.0    690
0.0    664
2.0    664
1.0    664
Name: count_dias_tweet, dtype: int64
2.0    684
3.0    678
0

In [49]:
#Move target to the end
cols = list (database.columns)
cols.pop (cols.index('Alvo'))
database = database[cols + ['Alvo']]
database
#database2 = database.copy()

,followers_count,friends_count,favourites_count,listed_count,url,location,default_profile,profile_banner_url,profile_image_url_https,age,in_following_followers,n_number_sname,size_sname,size_description,size_name,n_retweet_mean,n_favorite_mean,n_hashtags_mean,n_urls_mean,n_mentions_mean,media_size_twitter,dp_size_twitter,in_tweets_reply,n_tweet_dia,max_count_tweet_dia,count_dias_tweet,in_tweet_dia,Alvo
0,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,3.0,2.0,3.0,1.0,3.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,3.0,1.0,2.0,1
1,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,3.0,3.0,3.0,1.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,3.0,2.0,1.0,1
2,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,0.0,2.0,2.0,0.0,0.0,2.0,1.0,2.0,1
3,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,3.0,2.0,3.0,2.0,1.0,3.0,1.0,0.0,2.0,2.0,0.0,0.0,3.0,1.0,2.0,1
4,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,3.0,1.0,2.0,1
5,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0,2.0,1
6,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,3.0,2.0,1.0,1
7,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,3.0,2.0,3.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,3.0,2.0,1.0,1
8,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,1.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,3.0,1.0,0.0,1.0,2.0,2.0,2.0,1
9,0.0,0.0,0.0,0,0,1,0,0,1,5,0.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,0.0,2.0,2.0,0.0,0.0,3.0,1.0,2.0,1


In [139]:
#database['Alvo'] = database['Alvo'].map(lambda x: 1 if x =='Bot' else 'Gen')  

In [21]:
database.Alvo.value_counts()

1    1637
0    1045
Name: Alvo, dtype: int64

In [22]:
database.to_csv("users_tweets_bin.csv", sep=",")

In [151]:
records =[]


In [152]:
for i in range (0,2682):
    records.append([str(database.values[i,j]) for j in range(0, 28)])

In [166]:
association_rules = apriori(database, min_support=0.1, min_confidence=0.5, min_lift=1.3, min_length=2)

In [167]:
association_results = list(association_rules)

In [168]:
association_results

[RelationRecord(items=frozenset({'m', 'a'}), support=0.39285714285714285, ordered_statistics=[OrderedStatistic(items_base=frozenset({'a'}), items_add=frozenset({'m'}), confidence=0.5789473684210525, lift=1.4736842105263155), OrderedStatistic(items_base=frozenset({'m'}), items_add=frozenset({'a'}), confidence=1.0, lift=1.4736842105263157)]),
 RelationRecord(items=frozenset({'c', 'd'}), support=0.17857142857142858, ordered_statistics=[OrderedStatistic(items_base=frozenset({'c'}), items_add=frozenset({'d'}), confidence=0.625, lift=1.75), OrderedStatistic(items_base=frozenset({'d'}), items_add=frozenset({'c'}), confidence=0.5, lift=1.75)]),
 RelationRecord(items=frozenset({'n', 'c'}), support=0.2857142857142857, ordered_statistics=[OrderedStatistic(items_base=frozenset({'c'}), items_add=frozenset({'n'}), confidence=1.0, lift=1.3333333333333333)]),
 RelationRecord(items=frozenset({'c', 'o'}), support=0.2857142857142857, ordered_statistics=[OrderedStatistic(items_base=frozenset({'c'}), items